In [4]:
from pathlib import Path


In [1]:
import spacy

model = spacy.load("fr_dep_news_trf")

/home/lerner/anaconda3/envs/lightning/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import re

In [5]:
inclusive_root = Path('data/corpus-francais-inclusif/vfi/')

# Syntax (coordination) "toutes et tous"

In [31]:
def is_masc(token):
    gender = token.morph.get("Gender")
    return gender and gender[0] == "Masc" 

In [32]:
def keep_masc(a,b):
    if is_masc(a):
        return a
    return b

In [80]:
data = []
for path in inclusive_root.glob('*.txt'):
    with open(path, 'rt') as file:
        text = file.read()
    with open(path.with_suffix(".tag"),'rt') as file:
        print(re.findall(r'<Inclusif.+>(.+)</Inclusif>',file.read()))
    print(inclusif.findall(text))
    tokens = model(text)
    for sentence in tokens.sents:
        todo = []
        for token in sentence:
            if token.dep_=="ROOT" or token.pos_ in {"VERB","AUX"} or token.is_space:
                continue
            if token.lemma == token.head.lemma and token.text!=token.head.text:
                print(token,token.head,token.dep_,token.pos_,token.head.dep_,token.head.pos_,end=" ; ")   
                i = token.i-sentence.start
                j = token.head.i-sentence.start
                if j < i:
                    i, j = j, i
                todo.append((i, j, keep_masc(token.head,token)))
        if not todo:
            x_sentence = sub(sentence.text)
            if x_sentence != sentence.text:                
                datum=(sentence.text, x_sentence)
                print(datum)
                data.append(datum)   
            continue
        pi, pj, ptoken = todo[0]
        x_sentence = [sentence[:pi].text, ptoken.text]
        for i, j, token in todo[1:]:
            x_sentence.append(sentence[pj+1:i].text)
            x_sentence.append(token.text)
            pi, pj, ptoken = i, j, token
        x_sentence.append(sentence[pj+1:].text)
        x_sentence = " ".join(x_sentence)
        x_sentence = sub(x_sentence)
        datum=(sentence.text, x_sentence)
        print(datum)
        data.append(datum)            
    print("\n******************\n")
    #break

[]
[]
ceux celles conj PRON obl:arg PRON ; ("Je m'adresse à celles et ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main.", "Je m'adresse à ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main.")

******************

[]
[]

******************

['celles et ceux', 'Les Françaises et les Français', 'à toutes les jeunes Françaises et à tous les jeunes Français', 'des Américaines et des Américains', 'Aux femmes et aux hommes', 'des hommes et des femmes']
[]
ceux celles conj PRON conj PRON ; ceux celles conj PRON conj PRON ; ("Mes chers amis, celles et ceux qui sont à l'intérieur, celles et ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation leur permet de nous entendre...", "Mes chers amis, ceux qui sont à l'intérieur, ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était tr

In [51]:
data

[("Je m'adresse à celles et ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main.",
  "Je m'adresse à ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main."),
 ("Mes chers amis, celles et ceux qui sont à l'intérieur, celles et ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation leur permet de nous entendre...",
  "Mes chers amis, ceux qui sont à l'intérieur, ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation leur permet de nous entendre..."),
 ("\n\nLes Françaises et les Français m'ont placé en tête de dix candidats.",
  "\n\nLes Français m'ont placé en tête de dix candidats."),
 ("\n\nC'est pourquoi notre société doit être capable - et j'en prends l'engagement - d'offrir à toutes les jeunes Françaises et à tous les jeunes 

# Regex (écriture inclusive) "tous.tes"

In [68]:
seps = '[·.•]'
fem_suffix = r"[tr]?[rtnl]?(e|euse|ale|ice)"
inclusif_sing = re.compile(rf"({seps}{fem_suffix})|(\({fem_suffix}\))\b")
inclusif_plur = re.compile(rf"({seps}{fem_suffix}({seps}?s))|(\({fem_suffix}s\)|(\({fem_suffix}\)s)\b")

In [64]:
text="""Il s'agit de donner les moyens à chacun·e de satisfaire ses besoins mais aussi de s'émanciper dans une société plus démocratique, où la solidarité prendra le pas sur les égoïsmes.

Fin de l'élevage industriel, maltraitant pour les animaux et les salarié·e·s. Objectif 100% bio, développement des circuits courts.

Moratoire sur les dettes des agriculteurs·trices. Fonds public pour l'achat de foncier afin d'installer de jeunes agriculteurs·trices en coopérative.

Indemnités d'élu·e plafonnées au salaire médian. Révocabilité des élu·e·s.

Nous voulons une société solidaire et sans frontière, qui accueille tou·te·s les migrant·e·s

Droit de vote pour les étranger·e·s résidant en France"""

In [73]:
fix_s = re.compile(r"ss\b") # fixes erratic usages like "agriculteurS.ices" vs "agriculteur.ices"

In [76]:
def sub(text):
    text = inclusif_plur.sub('s', text)
    text = inclusif_sing.sub('', text)
    text = fix_s.sub('s', text)
    return text

In [78]:
print(sub(text))

Il s'agit de donner les moyens à chacun de satisfaire ses besoins mais aussi de s'émanciper dans une société plus démocratique, où la solidarité prendra le pas sur les égoïsmes.

Fin de l'élevage industriel, maltraitant pour les animaux et les salariés. Objectif 100% bio, développement des circuits courts.

Moratoire sur les dettes des agriculteurs. Fonds public pour l'achat de foncier afin d'installer de jeunes agriculteurs en coopérative.

Indemnités d'élu plafonnées au salaire médian. Révocabilité des élus.

Nous voulons une société solidaire et sans frontière, qui accueille tous les migrants

Droit de vote pour les étrangers résidant en France


In [69]:
print(inclusif_plur.sub('s', text))

Il s'agit de donner les moyens à chacun·e de satisfaire ses besoins mais aussi de s'émanciper dans une société plus démocratique, où la solidarité prendra le pas sur les égoïsmes.

Fin de l'élevage industriel, maltraitant pour les animaux et les salariés. Objectif 100% bio, développement des circuits courts.

Moratoire sur les dettes des agriculteurss. Fonds public pour l'achat de foncier afin d'installer de jeunes agriculteurss en coopérative.

Indemnités d'élu·e plafonnées au salaire médian. Révocabilité des élus.

Nous voulons une société solidaire et sans frontière, qui accueille tous les migrants

Droit de vote pour les étrangers résidant en France


In [70]:
print(inclusif_sing.sub('', text))

Il s'agit de donner les moyens à chacun de satisfaire ses besoins mais aussi de s'émanciper dans une société plus démocratique, où la solidarité prendra le pas sur les égoïsmes.

Fin de l'élevage industriel, maltraitant pour les animaux et les salarié·s. Objectif 100% bio, développement des circuits courts.

Moratoire sur les dettes des agriculteurss. Fonds public pour l'achat de foncier afin d'installer de jeunes agriculteurss en coopérative.

Indemnités d'élu plafonnées au salaire médian. Révocabilité des élu·s.

Nous voulons une société solidaire et sans frontière, qui accueille tou·s les migrant·s

Droit de vote pour les étranger·s résidant en France


In [66]:
inclusif.findall(text)

[('·e', 'e', '', '', ''),
 ('·e·s', 'e', '·s', '', ''),
 ('·trices', 'ice', 's', '', ''),
 ('·trices', 'ice', 's', '', ''),
 ('·e', 'e', '', '', ''),
 ('·e·s', 'e', '·s', '', ''),
 ('·te·s', 'e', '·s', '', ''),
 ('·e·s', 'e', '·s', '', ''),
 ('·e·s', 'e', '·s', '', '')]